In [1]:
# !pip install --upgrade kagglehub

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import os
import kagglehub
import pandas as pd
from PIL import Image

In [3]:
import kagglehub

def load_oxford_pet_dataset():
    return kagglehub.dataset_download("tanlikesmath/the-oxfordiiit-pet-dataset")

path_to_dataset = load_oxford_pet_dataset()
print(f"Dataset downloaded at: {path_to_dataset}")


100%|██████████| 1.48G/1.48G [00:51<00:00, 31.0MB/s]

Extracting model files...


Dataset downloaded at: /home/jovyan/.cache/kagglehub/datasets/tanlikesmath/the-oxfordiiit-pet-dataset/versions/1


In [4]:
def read_images_dataset(images_path):
    image_files = [f for f in os.listdir(images_path) if f.lower()
                   .endswith(('.jpg', '.jpeg', '.png'))]
    image_data = []

    for image_file in image_files:
        image_path = os.path.join(images_path, image_file)
        img = Image.open(image_path)

        image_data.append({
            "image_name": image_file,
            "image_path": image_path,
            "image_size": img.size
        })

    return pd.DataFrame(image_data)


In [8]:
dataset_path = "/home/jovyan/.cache/kagglehub/datasets/tanlikesmath/the-oxfordiiit-pet-dataset/versions/1/images"

In [9]:
oxford_pet_dataset = read_images_dataset(dataset_path)
oxford_pet_dataset.head(5)  # Show the first 5 rows

,image_name,image_path,image_size
0,Abyssinian_1.jpg,/home/jovyan/.cache/kagglehub/datasets/tanlike...,"(600, 400)"
1,Abyssinian_10.jpg,/home/jovyan/.cache/kagglehub/datasets/tanlike...,"(375, 500)"
2,Abyssinian_100.jpg,/home/jovyan/.cache/kagglehub/datasets/tanlike...,"(394, 500)"
3,Abyssinian_101.jpg,/home/jovyan/.cache/kagglehub/datasets/tanlike...,"(450, 313)"
4,Abyssinian_102.jpg,/home/jovyan/.cache/kagglehub/datasets/tanlike...,"(500, 465)"


In [10]:
def resize_image(image_path, dim):
    image = Image.open(image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')
    new_size = (dim, dim)
    resized_image = image.resize(new_size)
    return resized_image

In [11]:
def save_resize_oxford_pet_dataset(original_dataset):
    output_dir = "resized_images"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for index, row in original_dataset.iterrows():
        input_image_path = row['image_path']
        resized_image = resize_image(input_image_path, 224)

        output_image_path = os.path.join(output_dir, row['image_name'])
        resized_image.save(output_image_path)

    return output_dir


In [12]:
resized_dataset_dir_path = save_resize_oxford_pet_dataset(oxford_pet_dataset)
print(f"Resized images saved to: {resized_dataset_dir_path}")


Resized images saved to: resized_images


In [18]:
import os
import shutil

def organize_images_by_class(dataset_df, output_dir="resized_images"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in dataset_df.iterrows():
        class_name = row["image_name"].split("_")[0]  # Extract class name
        class_dir = os.path.join(output_dir, class_name)

        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        # Move the image into the class folder
        shutil.move(row["image_path"], os.path.join(class_dir, row["image_name"]))

    print(f"Images organized into class folders in: {output_dir}")

In [19]:
organize_images_by_class(oxford_pet_dataset)

Images organized into class folders in: resized_images


In [21]:
print(os.listdir("resized_images")[:10])

['Abyssinian_1.jpg', 'Abyssinian_10.jpg', 'Abyssinian_100.jpg', 'Abyssinian_101.jpg', 'Abyssinian_102.jpg', 'Abyssinian_103.jpg', 'Abyssinian_104.jpg', 'Abyssinian_105.jpg', 'Abyssinian_106.jpg', 'Abyssinian_107.jpg']


In [14]:
#!pip install timm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.1/436.1 kB 22.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm  # To use the TinyViT model

In [38]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Load the organized dataset
train_dataset = datasets.ImageFolder(root="resized_images", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"Total images in the training dataset: {len(train_dataset)}")


Total images in the training dataset: 7390


In [40]:
# Set up device for GPU/CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load TinyViT pretrained model
model = timm.create_model("tiny_vit_5m_224", pretrained=True, num_classes=len(train_dataset.classes))
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [41]:
# Split dataset: 80% training, 20% validation
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create train and validation DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 5912
Validation set size: 1478


In [42]:
# Training function
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct, total = 0, 0, 0
        
        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        # Validation loop
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        # Print epoch results
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {correct/total:.2%}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_correct/val_total:.2%}")
        
        model.train()  # Switch back to train mode

# Call the training function
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=5)


Epoch [1/5]
Train Loss: 2.5884, Train Acc: 57.48%
Val Loss: 1.6433, Val Acc: 85.39%
Epoch [2/5]
Train Loss: 1.1390, Train Acc: 88.09%
Val Loss: 0.7905, Val Acc: 89.45%
Epoch [3/5]
Train Loss: 0.5377, Train Acc: 93.81%
Val Loss: 0.4743, Val Acc: 92.15%
Epoch [4/5]
Train Loss: 0.2820, Train Acc: 96.84%
Val Loss: 0.3610, Val Acc: 92.63%
Epoch [5/5]
Train Loss: 0.1584, Train Acc: 98.58%
Val Loss: 0.3237, Val Acc: 91.41%


In [43]:
torch.save(model.state_dict(), "tinyvit_trained_model.pth")
print("Model saved as 'tinyvit_trained_model.pth'")

Model saved as 'tinyvit_trained_model.pth'
